d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

-sandbox
<img src="https://files.training.databricks.com/images/Apache-Spark-Logo_TM_200px.png" style="float: left: margin: 20px"/>

# Common Transformations

Apache Spark&trade; and Databricks&reg; allow you to manipulate data with built-in functions that accommodate common design patterns.

## In this lesson you:
* Apply built-in functions to manipulate data
* Define logic to handle null values
* Deduplicate a data set

## Audience
* Primary Audience: Data Engineers
* Additional Audiences: Data Scientists and Data Pipeline Engineers

## Prerequisites
* Web browser: Chrome
* Course: ETL Part 1 from <a href="https://academy.databricks.com/" target="_blank">Databricks Academy</a>

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Classroom-Setup & Classroom-Cleanup<br>

For each lesson to execute correctly, please make sure to run the **`Classroom-Setup`** cell at the start of each lesson (see the next cell) and the **`Classroom-Cleanup`** cell at the end of each lesson.

In [4]:
%run "./Includes/Classroom-Setup"

<iframe  
src="//fast.wistia.net/embed/iframe/xjbyksd137?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/xjbyksd137?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

-sandbox
### Transformations in ETL

The goal of transformations in ETL is to transform raw data in order to populate a data model.  The most common models are **relational models** and **snowflake (or star) schemas,** though other models such as query-first modeling also exist. Relational modeling entails distilling your data into efficient tables that you can join back together. A snowflake model is generally used in data warehousing where a fact table references any number of related dimension tables. Regardless of the model you use, the ETL approach is generally the same.

Transforming data can range in complexity from simply parsing relevant fields to handling null values without affecting downstream operations and applying complex conditional logic.  Common transformations include:<br><br>

* Normalizing values
* Imputing null or missing data
* Deduplicating data
* Performing database rollups
* Exploding arrays
* Pivoting DataFrames

<div><img src="https://files.training.databricks.com/images/eLearning/ETL-Part-2/data-models.png" style="height: 400px; margin: 20px"/></div>

-sandbox
### Built-In Functions

Built-in functions offer a range of performant options to manipulate data. This includes options familiar to:<br><br>

1. SQL users such as `.select()` and `.groupBy()`
2. Python, Scala and R users such as `max()` and `sum()`
3. Data warehousing options such as `rollup()` and `cube()`

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** For more depth on built-in functions, see  <a href="https://academy.databricks.com/collections/frontpage/products/dataframes" target="_blank">Getting Started with Apache Spark DataFrames course from Databricks Academy</a>.

### Normalizing Data

Normalizing refers to different practices including restructuring data in normal form to reduce redundancy, and scaling data down to a small, specified range. For this case, bound a range of integers between 0 and 1.

Start by taking a DataFrame of a range of integers

In [10]:
integerDF = spark.range(1000, 10000)

display(integerDF)

#  create a DF of a range of numbers from a to b


id
1000
1001
1002
1003
1004
1005
1006
1007
1008
1009


In [11]:

type(integerDF)


Out[14]: pyspark.sql.dataframe.DataFrame

-sandbox
To normalize these values between 0 and 1, subtract the minimum and divide by the maximum, minus the minimum.

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** <a href="http://spark.apache.org/docs/latest/api/python/pyspark.ml.html?highlight=minmaxscaler#pyspark.ml.feature.MinMaxScaler" target="_blank">Also see the built-in class `MinMaxScaler`</a>

In [13]:

from pyspark.sql.functions import col, max, min



colMin = integerDF.select(min("id")).first()[0]

colMax = integerDF.select(max("id")).first()[0]

normalizedIntegerDF = (integerDF
  .withColumn("normalizedValue", (col("id") - colMin) / (colMax - colMin) )
)

display(normalizedIntegerDF)


id,normalizedValue
1000,0.0
1001,1.1112345816201801E-4
1002,2.2224691632403602E-4
1003,3.33370374486054E-4
1004,4.4449383264807203E-4
1005,5.5561729081009E-4
1006,6.66740748972108E-4
1007,7.77864207134126E-4
1008,8.889876652961441E-4
1009,0.001000111123458162


In [14]:

# smallest value...

integerDF.select(min("id")).show()
# integerDF.select(min("id")


+-------+
min(id)|
+-------+
 1000|
+-------+

In [15]:

integerDF.select(min("id")).first()


Out[19]: Row(min(id)=1000)

In [16]:
integerDF.select(min("id")).first()[0]

Out[20]: 1000

<br><br><br>

-sandbox

### Imputing Null or Missing Data

Null values refer to unknown or missing data as well as irrelevant responses. Strategies for dealing with this scenario include:<br><br>

* **Dropping these records:** Works when you do not need to use the information for downstream workloads
* **Adding a placeholder (e.g. `-1`):** Allows you to see missing data later on without violating a schema
* **Basic imputing:** Allows you to have a "best guess" of what the data could have been, often by using the mean of non-missing data
* **Advanced imputing:** Determines the "best guess" of what data should be using more advanced strategies such as clustering machine learning algorithms or oversampling techniques 

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** <a href="http://spark.apache.org/docs/latest/api/python/pyspark.ml.html?highlight=imputer#pyspark.ml.feature.Imputer" target="_blank">Also see the built-in class `Imputer`</a>

Take a look at the following DataFrame, which has missing values.

In [20]:

corruptDF = spark.createDataFrame([
  (11, 66, 5),
  (12, 68, None),
  (1, None, 6),
  (2, 72, 7)], 
  ["hour", "temperature", "wind"]
)

display(corruptDF)


hour,temperature,wind
11,66,5
12,68,null
1,null,6
2,72,7


Drop any records that have null values.

In [22]:

corruptDroppedDF = corruptDF.dropna("any")

display(corruptDroppedDF)


hour,temperature,wind
11,66,5
2,72,7


Impute values with the mean.

In [24]:

corruptImputedDF = corruptDF.na.fill({"temperature": 68, "wind": 6})
display(corruptImputedDF)

# if you are missing, i will impute you with a temp col value of 68 or a wind col value of 6..



hour,temperature,wind
11,66,5
12,68,6
1,68,6
2,72,7


### Deduplicating Data

Duplicate data comes in many forms. The simple case involves records that are complete duplicates of another record. The more complex cases involve duplicates that are not complete matches, such as matches on one or two columns or "fuzzy" matches that account for formatting differences or other non-exact matches.

Take a look at the following DataFrame that has duplicate values.

In [27]:

duplicateDF = spark.createDataFrame([
  (15342, "Conor", "red"),
  (15342, "conor", "red"),
  (12512, "Dorothy", "blue"),
  (5234, "Doug", "aqua")], 
  ["id", "name", "favorite_color"]
)

display(duplicateDF)


id,name,favorite_color
15342,Conor,red
15342,conor,red
12512,Dorothy,blue
5234,Doug,aqua


Drop duplicates on `id` and `favorite_color`.

In [29]:
duplicateDedupedDF = duplicateDF.dropDuplicates(["id", "favorite_color"])
display(duplicateDedupedDF)

id,name,favorite_color
5234,Doug,aqua
12512,Dorothy,blue
15342,Conor,red


### Other Helpful Data Manipulation Functions

| Function    | Use                                                                                                                        |
|:------------|:---------------------------------------------------------------------------------------------------------------------------|
| `explode()` | Returns a new row for each element in the given array or map                                                               |
| `pivot()`   | Pivots a column of the current DataFrame and perform the specified aggregation                                             |
| `cube()`    | Create a multi-dimensional cube for the current DataFrame using the specified columns, so we can run aggregation on them   |
| `rollup()`  | Create a multi-dimensional rollup for the current DataFrame using the specified columns, so we can run aggregation on them |

In [31]:
# 

## Exercise 1: Deduplicating Data

A common ETL workload involves cleaning duplicated records that don't completely match up.  The source of the problem can be anything from user-generated content to schema evolution and data corruption.  Here, you match records and reduce duplicate records.

-sandbox
### Step 1: Import and Examine the Data

The file is sitting in `/mnt/training/dataframes/people-with-dups.txt`.

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** You have to deal with the header and delimiter.

In [34]:

%fs head /mnt/training/dataframes/people-with-dups.txt


[Truncated to first 65536 bytes]
firstName:middleName:lastName:gender:birthDate:salary:ssn
Emanuel:Wallace:Panton:M:1988-03-04:101255:935-90-7627
Eloisa:Rubye:Cayouette:F:2000-06-20:204031:935-89-9009
Cathi:Svetlana:Prins:F:2012-12-22:35895:959-30-7957
Mitchel:Andres:Mozdzierz:M:1966-05-06:55108:989-27-8093
Angla:Melba:Hartzheim:F:1938-07-26:13199:935-27-4276
Rachel:Marlin:Borremans:F:1923-02-23:67070:996-41-8616
Catarina:Phylicia:Dominic:F:1969-09-29:201021:999-84-8888
Antione:Randy:Hamacher:M:2004-03-05:271486:917-96-3554
Madaline:Shawanda:Piszczek:F:1996-03-17:183944:963-87-9974
Luciano:Norbert:Sarcone:M:1962-12-14:73069:909-96-1669
Newton:Jose:Piacente:M:1969-11-05:264422:967-61-8575
Gretta:Jennefer:Dipinto:F:1921-10-28:118497:907-49-2510
Jenni:Janella:Mcquiston:F:1969-07-11:137418:932-55-7164
Malena:Apryl:Kings:F:1980-01-08:98204:934-48-2334
Willy:Russell:Merker:M:1967-10-10:41026:992-83-4829
Jami:Allen:Mulkerin:F:1958-08-08:236024:934-86-4126
Fernando:Lowell:Zebell:M:1955-07-03:246516:951-57-6196
Garnet:Helaine:Edghill:F:1985-05-15:97125:979-42-4409
Siu:Cherrie:Lechelt:F:2012-07-24:148331:906-85-3202
Randal:Osvaldo:Torma:M:2013-12-17:275086:982-68-6906
Myles:Stuart:Miskell:M:1928-05-17:280726:992-89-2715
Conrad:Donnie:Piatkowski:M:1920-11-17:59935:916-33-7831
Darcel:Fleta:Czyrnik:F:1994-09-05:99641:995-95-8909
Juliane:Hallie:Vuillemot:F:1972-01-11:203415:950-17-2657
Cordia:Estella:Scheiber:F:1924-03-13:214853:965-74-4739
Bryon:Isiah:Nazario:M:2002-02-26:201478:963-38-1755
Alex:Gilbert:Trigueros:M:1990-12-30:271591:999-60-3232
Irwin:Rene:Zalesky:M:1931-06-20:122599:974-53-6387
Deandre:Brent:Bailor:M:1954-08-07:93429:920-78-9780
Cheree:Dorethea:Anspaugh:F:1985-01-17:278860:961-36-6578
Zachariah:Harris:Reemer:M:1924-12-16:238892:944-32-3013
See:Sharen:Howryla:F:1979-12-30:169570:925-12-1644
Mozella:Necole:Giannakopoulo:F:1993-08-24:40880:980-93-4844
David:Tracy:Delatrinidad:M:1932-01-26:85003:993-37-2066
Michel:Loren:Sandoz:M:1925-05-04:270046:948-56-4284
Graham:Denny:Apresa:M:1966-07-14:209118:928-67-3592
Kattie:Sammie:Ercek:F:2002-07-26:211993:996-32-1564
Shaunna:Kathlene:Amunrud:F:1965-07-01:234090:916-53-6216
Leeann:Madeleine:Clater:F:1976-10-26:100705:946-41-7003
Garry:Lino:Galm:M:1934-08-14:168769:994-29-1678
Shelly:Marhta:Krenzke:F:1915-05-08:270492:967-71-8565
Shantelle:Zoe:Spicher:F:1952-12-11:183419:989-79-8274
Yessenia:Lucina:Fred:F:1967-04-21:131376:990-95-7077
Del:Maynard:Dzurnak:M:1960-01-31:276319:938-46-9967
Maximina:Kirsten:Barranco:F:2004-11-05:257482:938-70-3631
Shasta:Bette:Licausi:F:2006-05-26:273311:929-84-1247
Wendell:Bud:Landi:M:2012-03-06:61215:987-95-9502
Sabine:Petronila:Lambertson:F:1937-03-31:274044:916-95-8546
Leonard:Filiberto:Sellberg:M:1982-05-25:264070:965-65-6038
Demetrice:Flossie:Schehr:F:1958-03-16:245090:990-42-7824
Reyes:Gilberto:Schlabs:M:1997-07-11:252679:906-83-6847
Bernard:Reggie:Coache:M:1960-06-23:53020:941-56-6401
Lavette:Elouise:Riedinger:F:1950-11-21:51371:913-81-9468
Edmund:Nicolas:Majocka:M:1974-05-26:298673:921-55-7816
Deana:Marcie:Malle:F:2011-06-24:15388:973-52-9173
Claudio:Leopoldo:Ealand:M:1916-01-14:158319:929-61-6825
Lucretia:Divina:Parnell:F:2011-05-30:41054:963-43-3135
Myrtis:Yaeko:Maciarello:F:1941-11-16:191903:996-58-7685
Cordie:Cara:Sheilds:F:2007-02-08:219449:950-98-5411
Michell:Corinne:Himmelspach:F:1968-02-27:154658:905-67-9613
Major:Shawn:Garay:M:1934-12-18:79152:904-49-5993
Andrew:Darnell:Kniceley:M:1966-09-02:25955:987-26-4880
Shasta:Floria:Messineo:F:2008-02-23:30339:925-83-2833
Madalene:Arline:Vanholland:F:1945-05-14:47734:977-68-6192
Clare:So:Bieker:F:1922-04-07:14603:922-40-9560
Audrey:Lorrine:Sprewell:F:1932-10-25:283164:997-53-7925
ARLEN:HAYDEN:CARVILL:M:1986-05-24:66754:918339442
Maybelle:Honey:Jolicoeur:F:1923-02-10:239772:902-46-1282
Darwin:Cedric:Gompf:M:1954-03-19:152991:926-88-9050
Jonelle:Maryam:Berkstresser:F:1944-11-17:253619:935-62-7974
Lynsey:Marcelina:Dagrella:F:1969-03-01:209989:978-42-7296
Robert:Jessie:Anaya:M:1922-09-30:136381:956-53-6

In [35]:

# reading in text file.... but i dont like the way i did this...

dupedDF = (spark.read
.format("csv")
.option("header","true")
.option("inferSchema","true") 
.option("delimiter", ":")
.load("/mnt/training/dataframes/people-with-dups.txt"))



In [36]:
# TEST - Run this cell to test your solution
cols = set(dupedDF.columns)

dbTest("ET2-P-02-01-01", 103000, dupedDF.count())
dbTest("ET2-P-02-01-02", True, "salary" in cols and "lastName" in cols)

print("Tests passed!")

Tests passed!

In [37]:

dupedDF.show(8, False)


+---------+----------+---------+------+-------------------+------+-----------+
firstName|middleName|lastName |gender|birthDate |salary|ssn |
+---------+----------+---------+------+-------------------+------+-----------+
Emanuel |Wallace |Panton |M |1988-03-04 00:00:00|101255|935-90-7627|
Eloisa |Rubye |Cayouette|F |2000-06-20 00:00:00|204031|935-89-9009|
Cathi |Svetlana |Prins |F |2012-12-22 00:00:00|35895 |959-30-7957|
Mitchel |Andres |Mozdzierz|M |1966-05-06 00:00:00|55108 |989-27-8093|
Angla |Melba |Hartzheim|F |1938-07-26 00:00:00|13199 |935-27-4276|
Rachel |Marlin |Borremans|F |1923-02-23 00:00:00|67070 |996-41-8616|
Catarina |Phylicia |Dominic |F |1969-09-29 00:00:00|201021|999-84-8888|
Antione |Randy |Hamacher |M |2004-03-05 00:00:00|271486|917-96-3554|
+---------+----------+---------+------+-------------------+------+-----------+
only showing top 8 rows

In [38]:
#
#
#

-sandbox
### Step 2: Add Columns to Filter Duplicates

Add columns following to allow you to filter duplicate values.  Add the following:

- `lcFirstName`: first name lower case
- `lcLastName`: last name lower case
- `lcMiddleName`: middle name lower case
- `ssnNums`: social security number without hyphens between numbers

Save the results to `dupedWithColsDF`.

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** Use the Spark function `lower()`

In [40]:

from pyspark.sql.functions import col, lower


dupedWithColsDF = (dupedDF.withColumn("lcFirstName", lower(col("firstName")))
                   .withColumn("lcLastName", lower(col("lastName")))
                   .withColumn("lcMiddleName", lower(col("middleName")))
                   .withColumn("ssnNums", lower(col("ssn"))))



In [41]:

dupedWithColsDF.show(10)


+---------+----------+---------+------+-------------------+------+-----------+-----------+----------+------------+-----------+
firstName|middleName| lastName|gender| birthDate|salary| ssn|lcFirstName|lcLastName|lcMiddleName| ssnNums|
+---------+----------+---------+------+-------------------+------+-----------+-----------+----------+------------+-----------+
 Emanuel| Wallace| Panton| M|1988-03-04 00:00:00|101255|935-90-7627| emanuel| panton| wallace|935-90-7627|
 Eloisa| Rubye|Cayouette| F|2000-06-20 00:00:00|204031|935-89-9009| eloisa| cayouette| rubye|935-89-9009|
 Cathi| Svetlana| Prins| F|2012-12-22 00:00:00| 35895|959-30-7957| cathi| prins| svetlana|959-30-7957|
 Mitchel| Andres|Mozdzierz| M|1966-05-06 00:00:00| 55108|989-27-8093| mitchel| mozdzierz| andres|989-27-8093|
 Angla| Melba|Hartzheim| F|1938-07-26 00:00:00| 13199|935-27-4276| angla| hartzheim| melba|935-27-4276|
 Rachel| Marlin|Borremans| F|1923-02-23 00:00:00| 67070|996-41-8616| rachel| borremans| marlin|996-41-8616|
 Catarina| Phylicia| Dominic| F|1969-09-29 00:00:00|201021|999-84-8888| catarina| dominic| phylicia|999-84-8888|
 Antione| Randy| Hamacher| M|2004-03-05 00:00:00|271486|917-96-3554| antione| hamacher| randy|917-96-3554|
 Madaline| Shawanda| Piszczek| F|1996-03-17 00:00:00|183944|963-87-9974| madaline| piszczek| shawanda|963-87-9974|
 Luciano| Norbert| Sarcone| M|1962-12-14 00:00:00| 73069|909-96-1669| luciano| sarcone| norbert|909-96-1669|
+---------+----------+---------+------+-------------------+------+-----------+-----------+----------+------------+-----------+
only showing top 10 rows

In [42]:
# TEST - Run this cell to test your solution
cols = set(dupedWithColsDF.columns)

dbTest("ET2-P-02-02-01", 103000, dupedWithColsDF.count())
dbTest("ET2-P-02-02-02", True, "lcFirstName" in cols and "lcLastName" in cols)

print("Tests passed!")

Tests passed!

### Step 3: Deduplicate the Data

Deduplicate the data by dropping duplicates of all records except for the original names (first, middle, and last) and the original `ssn`.  Save the result to `dedupedDF`.  Drop the columns you added in step 2.

In [44]:

columns_dup_to_use = ["gender", "birthDate", "salary", "lcFirstName", "lcLastName", "lcMiddleName", "ssnNums"]

dedupedDF = dupedWithColsDF.dropDuplicates(columns_dup_to_use)

dedupedDF = dedupedDF.drop("lcFirstName", "lcLastName", "lcMiddleName", "ssnNums")\

display(dedupedDF)


firstName,middleName,lastName,gender,birthDate,salary,ssn
ABE,HERSCHEL,PIDCOCK,M,1957-08-25T00:00:00.000+0000,218209,974393395
Abraham,Alva,Gerstein,M,1994-05-28T00:00:00.000+0000,261603,986-11-2405
Adah,Leota,Gearin,F,1918-06-11T00:00:00.000+0000,136140,941-69-2089
Adam,Erich,Gilbertson,M,1918-06-19T00:00:00.000+0000,126431,940-67-6657
Adeline,Elfriede,Vacanti,F,1915-01-11T00:00:00.000+0000,60230,927-62-5100
Adina,Cythia,Difiora,F,1944-12-11T00:00:00.000+0000,282060,943-38-4882
Adolfo,Newton,Klock,M,1954-04-08T00:00:00.000+0000,35759,978-24-2995
Adolph,Dominique,Dosier,M,2014-09-06T00:00:00.000+0000,50967,912-48-5632
Adrian,Kraig,Kirch,M,1950-08-09T00:00:00.000+0000,276857,972-37-7565
Aja,Jamey,Rohrbach,F,1964-11-28T00:00:00.000+0000,243157,979-26-3630


In [45]:
# TEST - Run this cell to test your solution
cols = set(dedupedDF.columns)

dbTest("ET2-P-02-03-01", 100000, dedupedDF.count())
dbTest("ET2-P-02-03-02", 7, len(cols))

print("Tests passed!")

--------------------------------------------------------------------------- 
 AssertionError Traceback (most recent call last)
 <command-3933795589709111> in <module> 
 2 cols = set ( dedupedDF . columns ) 
 3 
 ----> 4 dbTest ( "ET2-P-02-03-01" , 100000 , dedupedDF . count ( ) ) 
 5 dbTest ( "ET2-P-02-03-02" , 7 , len ( cols ) ) 
 6 

 <command-3933795589708537> in dbTest (id, expected, result) 
 18 daLogger . logEvent ( id , f"{eventId}\nDBTest Assertion\n{status}\n1" ) 
 19 
 ---> 20 assert evaluation , f"{result} does not equal expected {expected}" 
 21 
 22 ############################################# 

 AssertionError : 102961 does not equal expected 100000

## Review
**Question:** What built-in functions are available in Spark?  
**Answer:** Built-in functions include SQL functions, common programming language primitives, and data warehousing specific functions.  See the Spark API Docs for more details. (<a href="http://spark.apache.org/docs/latest/api/python/index.html" target="_blank">Python</a> or <a href="http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.package" target="_blank">Scala</a>).

**Question:** What's the best way to handle null values?  
**Answer:** The answer depends largely on what you hope to do with your data moving forward. You can drop null values or impute them with a number of different techniques.  For instance, clustering your data to fill null values with the values of nearby neighbors often gives more insight to machine learning models than using a simple mean.

**Question:** What are potential challenges of deduplicating data and imputing null values?  
**Answer:** Challenges include knowing which is the correct record to keep and how to define logic that applies to the root cause of your situation. This decision making process depends largely on how removing or imputing data will affect downstream operations like database queries and machine learning workloads. Knowing the end application of the data helps determine the best strategy to use.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Classroom-Cleanup<br>

Run the **`Classroom-Cleanup`** cell below to remove any artifacts created by this lesson.

In [48]:
%run "./Includes/Classroom-Cleanup"

## Next Steps

Start the next lesson, [User Defined Functions]($./ETL2 03 - User Defined Functions ).

## Additional Topics & Resources

**Q:** How can I do ACID transactions with Spark?  
**A:** ACID compliance refers to a set of properties of database transactions that guarantee the validity of you data.  <a href="https://databricks.com/product/databricks-delta" target="_blank">Databricks Delta</a> is an ACID compliant solution to transactionality with Spark workloads.

**Q:** How can I handle more complex conditional logic in Spark?  
**A:** You can handle more complex if/then conditional logic using the `when()` function and its `.otherwise()` method.

**Q:** How can I handle data warehousing functions like rollups?  
**A:** Spark allows for rollups and cubes, which are common in star schemas, using the `rollup()` and `cube()` functions.

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>